# 🏅 LEMMA IMO Substitution Model Training

This notebook trains a DistilBERT model to predict useful substitutions for IMO problems.

**GPU Quota**: 30 hours/week (P100)
**Estimated Training Time**: 1-2 hours

## Setup
1. Enable GPU: Settings → Accelerator → GPU P100
2. Run all cells
3. Download the ONNX model at the end

In [ ]:
# Install dependencies
!pip install transformers datasets scikit-learn onnx onnxruntime -q

In [ ]:
import json
import torch
import numpy as np
from pathlib import Path
from typing import List, Dict
from torch.utils.data import Dataset, DataLoader
from transformers import (
    DistilBertTokenizer,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from sklearn.preprocessing import MultiLabelBinarizer

print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 1. Substitution Vocabulary

These are the substitutions the model will learn to predict.

In [ ]:
# Substitution vocabulary - what the model will predict
SUBSTITUTION_VOCAB = [
    "x = 0",
    "y = 0",
    "x = y",
    "x = 1",
    "y = 1",
    "a = b = c = 1",
    "abc = 1 constraint",
    "Apply AM-GM",
    "Apply Cauchy-Schwarz",
    "Assume f is linear",
    "Assume f is injective",
    "Assume f is monotonic",
    "Check small cases",
    "Use modular arithmetic",
    "Homogenize",
    "WLOG assume ordering",
    "Substitute c = 1/(ab)",
    "y = f(x)",
    "x = -y",
    "Consider p = 2 separately",
]

print(f"Vocabulary size: {len(SUBSTITUTION_VOCAB)} substitutions")

## 2. Training Data

Annotated IMO problems with substitutions.

In [ ]:
# Training data - annotated IMO problems
TRAINING_DATA = [
    {
        "problem_text": "Find all functions f: ℝ⁺ → ℝ⁺ such that for every x ∈ ℝ⁺, there is exactly one y ∈ ℝ⁺ satisfying xf(y) + yf(x) ≤ 2.",
        "substitutions": ["x = y", "x = 1"]
    },
    {
        "problem_text": "Find all functions f: ℤ → ℤ such that for all integers a, b: f(2a) + 2f(b) = f(f(a + b)).",
        "substitutions": ["x = 0", "y = 0", "x = y"]
    },
    {
        "problem_text": "Find all functions f: ℝ → ℝ such that f(f(x)f(y)) + f(x + y) = f(xy) for all x, y ∈ ℝ.",
        "substitutions": ["x = 0", "y = 0", "x = 1", "y = 1"]
    },
    {
        "problem_text": "Find all functions f: ℝ → ℝ that satisfy f(x + f(x + y)) + f(xy) = x + f(x + y) + yf(x).",
        "substitutions": ["y = 0", "x = 0", "Assume f is linear"]
    },
    {
        "problem_text": "Find all functions f: ℝ → ℝ such that for all x, y ∈ ℝ: f(⌊x⌋y) = f(x)⌊f(y)⌋.",
        "substitutions": ["x = 0", "y = 0", "x = 1"]
    },
    {
        "problem_text": "Prove that for positive reals a, b, c with abc = 1: (a-1+1/b)(b-1+1/c)(c-1+1/a) ≤ 1.",
        "substitutions": ["abc = 1 constraint", "a = b = c = 1", "Apply AM-GM"]
    },
    {
        "problem_text": "Prove that for positive reals a, b, c: a/√(a² + 8bc) + b/√(b² + 8ca) + c/√(c² + 8ab) ≥ 1.",
        "substitutions": ["a = b = c = 1", "Apply Cauchy-Schwarz", "Homogenize"]
    },
    {
        "problem_text": "Find all positive integers n for which n divides 2^n + 1.",
        "substitutions": ["Check small cases", "Use modular arithmetic"]
    },
    {
        "problem_text": "Find all pairs (a,b) of positive integers such that gcd(a^n + b, b^n + a) is eventually constant.",
        "substitutions": ["Check small cases", "Use modular arithmetic", "Consider p = 2 separately"]
    },
    {
        "problem_text": "Find all functions f: ℕ → ℕ such that f(m + f(n)) = f(f(m)) + f(n) for all m, n.",
        "substitutions": ["x = 0", "y = 0", "y = f(x)", "Assume f is injective"]
    },
    {
        "problem_text": "Let a, b, c be positive reals with a + b + c = 3. Prove that a² + b² + c² ≥ 3.",
        "substitutions": ["Apply Cauchy-Schwarz", "a = b = c = 1"]
    },
    {
        "problem_text": "Find all functions f: ℚ → ℚ such that f(x + f(y)) = f(x) + y for all x, y ∈ ℚ.",
        "substitutions": ["x = 0", "y = 0", "Assume f is linear"]
    },
    {
        "problem_text": "Let a, b, c be positive reals with abc = 1. Prove that 1/(a³(b+c)) + 1/(b³(c+a)) + 1/(c³(a+b)) ≥ 3/2.",
        "substitutions": ["abc = 1 constraint", "a = b = c = 1", "Apply AM-GM", "WLOG assume ordering"]
    },
    {
        "problem_text": "Find all functions f: ℝ → ℝ such that f(x²) - f(y²) = (f(x) + y)(x - f(y)).",
        "substitutions": ["x = 0", "y = 0", "x = y", "x = -y"]
    },
    {
        "problem_text": "Prove that for all positive integers n, the number n^(n+1) + (n+1)^n is divisible by gcd(n, n+1) + 1.",
        "substitutions": ["Check small cases", "Use modular arithmetic"]
    },
]

print(f"Training examples: {len(TRAINING_DATA)}")

## 3. Dataset Class

In [ ]:
class SubstitutionDataset(Dataset):
    def __init__(self, problems, tokenizer, mlb, max_length=256):
        self.tokenizer = tokenizer
        self.mlb = mlb
        self.max_length = max_length
        self.data = problems
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        
        # Tokenize
        encoding = self.tokenizer(
            item['problem_text'],
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt',
        )
        
        # Multi-label target
        labels = self.mlb.transform([item['substitutions']])
        
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(labels.squeeze(), dtype=torch.float),
        }

## 4. Initialize Model & Tokenizer

In [ ]:
# Initialize
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
mlb = MultiLabelBinarizer(classes=SUBSTITUTION_VOCAB)
mlb.fit([SUBSTITUTION_VOCAB])

# Create dataset
dataset = SubstitutionDataset(TRAINING_DATA, tokenizer, mlb)

# Split into train/val (80/20)
train_size = int(0.8 * len(dataset))
train_dataset = torch.utils.data.Subset(dataset, range(train_size))
val_dataset = torch.utils.data.Subset(dataset, range(train_size, len(dataset)))

print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}")

In [ ]:
# Load model
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=len(SUBSTITUTION_VOCAB),
    problem_type='multi_label_classification',
)

print(f"Model loaded with {len(SUBSTITUTION_VOCAB)} output labels")

## 5. Training

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = (torch.sigmoid(torch.tensor(pred.predictions)) > 0.5).numpy()
    
    # Per-sample exact match
    exact_match = np.all(preds == labels, axis=1).mean()
    
    # Per-label accuracy
    label_accuracy = (preds == labels).mean()
    
    return {
        'exact_match': exact_match,
        'label_accuracy': label_accuracy,
    }

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=20,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='label_accuracy',
    greater_is_better=True,
    fp16=torch.cuda.is_available(),  # Mixed precision if GPU available
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
# Train!
print("Starting training...")
trainer.train()
print("Training complete!")

## 6. Test the Model

In [ ]:
def predict_substitutions(text, top_k=3):
    """Predict substitutions for a problem."""
    model.eval()
    
    inputs = tokenizer(
        text, 
        return_tensors='pt', 
        max_length=256, 
        truncation=True, 
        padding='max_length'
    )
    
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
        model.cuda()
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    probs = torch.sigmoid(outputs.logits).squeeze().cpu().numpy()
    
    # Get top-k
    top_indices = probs.argsort()[-top_k:][::-1]
    results = [(SUBSTITUTION_VOCAB[i], float(probs[i])) for i in top_indices]
    
    return results

In [ ]:
# Test on IMO 2022 P3
test_problem = "Find all functions f: ℝ⁺ → ℝ⁺ such that for every x ∈ ℝ⁺, there is exactly one y ∈ ℝ⁺ satisfying xf(y) + yf(x) ≤ 2."

print("Problem:", test_problem[:100] + "...")
print("\nPredicted substitutions:")
for sub, prob in predict_substitutions(test_problem, top_k=5):
    print(f"  {prob:.2%} - {sub}")

In [ ]:
# Test on an algebra problem
test_algebra = "Let a, b, c be positive reals with abc = 1. Prove that a + b + c >= 3."

print("Problem:", test_algebra)
print("\nPredicted substitutions:")
for sub, prob in predict_substitutions(test_algebra, top_k=5):
    print(f"  {prob:.2%} - {sub}")

## 7. Export to ONNX

In [ ]:
# Save PyTorch model first
model_dir = Path('./final_model')
model_dir.mkdir(exist_ok=True)

trainer.save_model(str(model_dir))
tokenizer.save_pretrained(str(model_dir))

# Save vocab
with open(model_dir / 'label_vocab.json', 'w') as f:
    json.dump(SUBSTITUTION_VOCAB, f)

print(f"Model saved to {model_dir}")

In [ ]:
# Export to ONNX
model.eval()
model.cpu()

# Dummy input
dummy_input = tokenizer(
    "Find all functions f such that f(x+y) = f(x) + f(y)",
    return_tensors='pt',
    max_length=256,
    truncation=True,
    padding='max_length'
)

onnx_path = model_dir / 'substitution_model.onnx'

torch.onnx.export(
    model,
    (dummy_input['input_ids'], dummy_input['attention_mask']),
    str(onnx_path),
    input_names=['input_ids', 'attention_mask'],
    output_names=['logits'],
    dynamic_axes={
        'input_ids': {0: 'batch'},
        'attention_mask': {0: 'batch'},
        'logits': {0: 'batch'},
    },
    opset_version=14,
)

print(f"ONNX model saved to {onnx_path}")

In [ ]:
# Verify ONNX model
import onnxruntime as ort

session = ort.InferenceSession(str(onnx_path))
result = session.run(
    None,
    {
        'input_ids': dummy_input['input_ids'].numpy(),
        'attention_mask': dummy_input['attention_mask'].numpy(),
    }
)

print(f"✅ ONNX verification passed! Output shape: {result[0].shape}")

## 8. Download Model

Download these files for LEMMA integration:
- `final_model/substitution_model.onnx` - For Rust inference
- `final_model/label_vocab.json` - Substitution vocabulary

In [ ]:
# Create zip for easy download
import shutil

shutil.make_archive('lemma_substitution_model', 'zip', model_dir)
print("Created: lemma_substitution_model.zip")
print("\nDownload this file and extract to: c:\\rust\\math-monster\\model\\")

In [ ]:
# List files for download
print("Files to download:")
for f in model_dir.iterdir():
    size_kb = f.stat().st_size / 1024
    print(f"  {f.name}: {size_kb:.1f} KB")